<a href="https://colab.research.google.com/github/mdias23i/DE-DataProcessing/blob/main/spark_streaming/examples/example_3_api_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [1]:
%pip install pyspark

In [145]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()
sc = spark.sparkContext

In [142]:
!rm -rf /content/landing
!rm -rf /content/bronze
!mkdir -p /content/landing

## Simulate producer:
- extract data from API
- store data as json in the lake
- run task async

In [146]:
import requests
from pyspark.sql.types import *
import json
import datetime
import asyncio

async def ingest_from_api(url: str, table: str, schema: StructType = None):
  response = requests.get(url)
  timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
  if response.status_code == 200:
    data = response.json()
    with open(f"/content/landing/{table}_{int(timestamp)}.json", "w") as f:
        json.dump(data, f)

async def producer(loop: int, interval_time: int):
  for i in range(loop):
    await ingest_from_api("https://api.carrismetropolitana.pt/vehicles", "vehicles")
    await ingest_from_api("https://api.carrismetropolitana.pt/lines", "lines")
    await asyncio.sleep(interval_time)

async def main():
  asyncio.create_task(producer(10, 30))

await main()

- Read from /content/landing as streaming
- store data in memory (for testing)
- store data in the bronze layer

In [147]:
from pyspark.sql.types import *

vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])

stream = spark.readStream.format("json").schema(vehicle_schema).load("/content/landing/vehicles*")

dedup = stream.dropDuplicates()

In [148]:
# using memory for testing
try:
  if query.isActive:
    query.stop()
except:
  pass

query = (dedup.writeStream.format("memory").option("queryName", "vehicles").start())

In [152]:
query.lastProgress

{'id': '64217b58-2e1e-4080-8a9c-32dc31b4c160',
 'runId': '0bc9375a-dca3-4771-aee3-a1cdb7e6f563',
 'name': 'vehicles',
 'timestamp': '2024-11-30T10:45:24.550Z',
 'batchId': 0,
 'numInputRows': 1138,
 'inputRowsPerSecond': 0.0,
 'processedRowsPerSecond': 93.0879345603272,
 'durationMs': {'addBatch': 12105,
  'commitOffsets': 29,
  'getBatch': 9,
  'latestOffset': 44,
  'queryPlanning': 9,
  'triggerExecution': 12225,
  'walCommit': 27},
 'stateOperators': [{'operatorName': 'dedupe',
   'numRowsTotal': 666,
   'numRowsUpdated': 666,
   'allUpdatesTimeMs': 3120,
   'numRowsRemoved': 0,
   'allRemovalsTimeMs': 0,
   'commitTimeMs': 5587,
   'memoryUsedBytes': 337304,
   'numRowsDroppedByWatermark': 0,
   'numShufflePartitions': 200,
   'numStateStoreInstances': 200,
   'customMetrics': {'loadedMapCacheHitCount': 0,
    'loadedMapCacheMissCount': 0,
    'numDroppedDuplicateRows': 472,
    'stateOnCurrentVersionSizeBytes': 308504}}],
 'sources': [{'description': 'FileStreamSource[file:/conten

In [135]:
query.isActive

True

In [153]:
query.stop()

In [154]:
spark.sql("select * from vehicles").show()

+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|bearing|            block_id|current_status|      id|      lat|line_id|      lon|pattern_id|route_id|schedule_relationship|    shift_id|    speed|stop_id|          timestamp|             trip_id|
+-------+--------------------+--------------+--------+---------+-------+---------+----------+--------+---------------------+------------+---------+-------+-------------------+--------------------+
|    254|           2_2609-21|    STOPPED_AT| 41|1254|38.755657|   1721|-9.268077|  1721_0_1|  1721_0|            SCHEDULED|        2609|      0.0| 170931|2024-11-30 10:44:38|1721_0_1_1000_102...|
|      0|           2_2613-21|    STOPPED_AT| 41|1811|38.760933|   1226|-9.261893|  1226_0_2|  1226_0|            SCHEDULED|        2613|      0.0| 170877|2024-11-30 10:45:01|1226_0_2_1030_105...|
|    350|202411

In [27]:
!rm -rf /content/bronze

In [155]:
from pyspark.sql.functions import *

# watermark is necessary because of the aggregation
transformed = stream.withWatermark("timestamp", "60 seconds")
agg = (transformed
       .groupBy(window(transformed.timestamp, "5 minutes"), col("current_status"))
       .agg(min(transformed.timestamp).alias("init_timestamp"), count("*").alias("count")))

def insert_vehicles(df, batch_id):
  #df2 = df.groupBy("window").pivot("current_status").sum("count")
  df.write.format("parquet").mode("append").save("/content/bronze/vehicles")

# using memory for testing
query2 = (agg
          .writeStream
          .outputMode("append")
          .foreachBatch(insert_vehicles)
          .option("checkpointLocation", "/content/bronze/checkpoint")
          .trigger(processingTime='20 seconds')
          .start())

In [159]:
query2.isActive

True

In [162]:
spark.read.format("parquet").load("/content/bronze/vehicles/*").show(100, False)

+------------------------------------------+--------------+-------------------+-----+
|window                                    |current_status|init_timestamp     |count|
+------------------------------------------+--------------+-------------------+-----+
|{2024-11-30 10:40:00, 2024-11-30 10:45:00}|IN_TRANSIT_TO |2024-11-30 10:43:47|626  |
|{2024-11-30 10:40:00, 2024-11-30 10:45:00}|INCOMING_AT   |2024-11-30 10:43:18|190  |
|{2024-11-30 10:40:00, 2024-11-30 10:45:00}|STOPPED_AT    |2024-11-30 10:43:20|348  |
+------------------------------------------+--------------+-------------------+-----+



## Report
- show vehicles by status in 5-min window time
- one line per window time

In [163]:
def pivot_data(df: DataFrame):
  result = df.orderBy("init_timestamp").groupBy("window").pivot("current_status").sum("count")
  result.show(100, False)

df = spark.read.format("parquet").load("/content/bronze/vehicles/*")
pivot_data(df)

+------------------------------------------+-----------+-------------+----------+
|window                                    |INCOMING_AT|IN_TRANSIT_TO|STOPPED_AT|
+------------------------------------------+-----------+-------------+----------+
|{2024-11-30 10:40:00, 2024-11-30 10:45:00}|190        |626          |348       |
+------------------------------------------+-----------+-------------+----------+



In [164]:
query2.stop()